In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import submission
from tqdm import tqdm
import pandas as pd
import google.generativeai as genai
import time

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/ericchen/opt/anaconda3/envs/Genai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Part 0: Signature and API Key

First step, have the signature function return your first and last name as a string. The below code should make sure that your signature function is implemented bug free.

In [4]:
signature = submission.signature()
assert isinstance(signature, str), 'signature must be a string'
print(f'signature: {signature}')

signature: Yifei Chen


Get your API key from [this link](https://ai.google.dev/gemini-api/docs) by clicking on the `Get a Gemini API Key` button and following the instructions.

Once you have your API key, put it in the `api_key()` function in the `submission.py` file.

```python
def api_key():
    """
    Return your api key as a string.
    """
    key = "api_key_here"
    return key
```

In [5]:
api_key = submission.api_key()
assert api_key is not None, 'api_key must be a string'

# Math Problems

GSMK8 is a data set consisting of grade school math questions. Today, many language models are able to answer those questions with near perfect accuracy because they either have been trained on the data set, or have been prompted to solve the questions in a specific way.

One thing that GPT 4o still has trouble doing is with math problems of large numbers. However, most arithmetic problems follow a very simple and algorithmic process.

The purpose of this assignment is to get you to think about how you can prompt the model to produce an answer to a large multiplication question that follows the two rules:
1. The answer must be correct.
2. The answer must follow a specific format, namely, answers must end with a number, with no other text or symbols, and the answer should be delimited by a `#### answer: ` tag.

For example, consider the answer:

`What is 15348 * 1256?. #### answer: 193107488`

This answer is correct and follows the format, where the answer is delimited by a `#### answer: ` tag and ends with a number without any other text or symbols.

Your job is to write a prompt in the `submission.py` file that will produce the desired answer.

## Format string

Suppose we have the following string in Python called `prompt`:

```python
prompt = """
    {question}
    #### answer: <answer>
"""
```
Notice that `{question}` is a placeholder for the question surrounded by the curly braces. Python has a string method called `.format()` that allows you to pass in variables to the string. In this case, `question` is a variable that format will replace with the actual question. For example, if we have the following question:

`What is 15348 * 1256?`

Then calling `prompt.format(question='What is 15348 * 1256?')` will return the following string:

```python
"""
What is 15348 * 1256?
#### answer: <answer>
"""
```

This is because the string `prompt` knows that `question` is a variable and will look for a parameter called `question` when the `.format()` method is called.

Use this format to write your prompt in the `submission.py` file. You are free to add any additional context to the prompt that you think will help the model answer the question, but the `def prompt()` function will only take in a question as a parameter. See the comments in the `submission.py` file for more information.

## Dataset

We provide a dataset of 100 multiplication questions below.

Your grade will be based on the accuracy of your prompt when we run it on a different data set of 100 questions that may involve more complex large number math problems. Thus you cannot hard code the answer in your prompt. Feel free to experiment with your own questions to see how well your prompt works.

## Submission

You will only submit the `submission.py` file to gradescope. If you want to test your prompt, you can run the following code. 

Be sure you complete the api_key() function or you will not be able to get a score.

## Grading

We will run your prompt on a different dataset of 25 of the same type of questions. Your score will be the accuracy of your prompt on this dataset.


- Gradescope Accuracy >= 0.85 = Full score
- Gradescope Accuracy >= 0.7 = 0.75 score
- Gradescope Accuracy >= 0.6 = 0.5 score
- Gradescope Accuracy > 0.5 = 0.25 score
- Else = 0.0 score

Your submission should take less than 4 minutes to grade. You are rate limited to 15 requests per minute, per requirements set by Gemini.

The following code below will load your dataset and run forward for 25 questions. 

In [6]:
submission.naive_prompt("What is the context?")
question = "What is 1803 * 795?"
context1 = "You are a highly accurate calculator. Solve the math problem below and provide the final answer in the correct format. #### answer: <answer>"
answer = "Your response must end with '#### answer: <answer>', where <answer> is the calculated number. The final answer must be a number, either an integer or a float. Do not include any text, symbols, or units after the number."
prompt = """
        
        <context1>
        {context1}
        </context1>

        <question>
        {question}
        </question>

        <answer>
        {answer}
        </answer>
    """
    
prompt = prompt.format(question=question, context1=context1, answer=answer)
display(prompt)

"\n        \n        <context1>\n        You are a highly accurate calculator. Solve the math problem below and provide the final answer in the correct format. #### answer: <answer>\n        </context1>\n\n        <question>\n        What is 1803 * 795?\n        </question>\n\n        <answer>\n        Your response must end with '#### answer: <answer>', where <answer> is the calculated number. The final answer must be a number, either an integer or a float. Do not include any text, symbols, or units after the number.\n        </answer>\n    "

In [7]:
# run this to load the dataset
df = pd.read_csv('train.csv')
questions = df['question'].tolist()
truth = df['answer'].tolist()

In [8]:
api_key = submission.api_key()
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-1.5-flash")
def get_accuracy(n=20):
    answers = []

    for question in tqdm(questions[:n]):
        splitted_text = model.generate_content(submission.prompt(question)).text.lower().split('answer: ')
        gpt_answer = splitted_text[1].strip()
        print(gpt_answer)
        answers.append(float(gpt_answer))
        print(splitted_text)
        time.sleep(4)
    return answers
answers = get_accuracy(n=25)
print(answers)

  0%|          | 0/25 [00:00<?, ?it/s]

1433385
['calculation steps:\n\n1. multiply 1803 by 5 (the ones digit of 795): 1803 * 5 = 9015\n\n2. multiply 1803 by 90 (the tens digit of 795): 1803 * 90 = 162270\n\n3. multiply 1803 by 700 (the hundreds digit of 795): 1803 * 700 = 1262100\n\n4. add the three intermediate results together: 9015 + 162270 + 1262100 = 1433385\n\n\nverification:\n\nwe can verify the result by dividing 1433385 by 795.  if the result is 1803, our multiplication was correct.\n\n1433385 / 795 = 1803\n\n#### ', '1433385\n']


  4%|▍         | 1/25 [00:05<02:21,  5.91s/it]

3403660
['calculation steps:\n\n1. multiply 6890 by 4 (the ones digit of 494): 6890 * 4 = 27560\n\n2. multiply 6890 by 9 (the tens digit of 494): 6890 * 90 = 620100\n\n3. multiply 6890 by 4 (the hundreds digit of 494): 6890 * 400 = 2756000\n\n4. add the results from steps 1, 2, and 3: 27560 + 620100 + 2756000 = 3403660\n\nverification:\n\nto verify, we can perform the division: 3403660 / 494 = 6890 (approximately). the slight difference might arise from rounding errors if division is not exact.\n\n\n#### ', '3403660\n']


  8%|▊         | 2/25 [00:11<02:13,  5.80s/it]

...

-54078036
['calculation steps:\n\n1. **identify the operation:** the question asks for the difference between two numbers, indicating subtraction.\n\n2. **perform subtraction:** we subtract 97075874 from 42997838.\n\n   42997838 - 97075874 = -54078036\n\n\n3. **verify the result (using addition):**  to check our subtraction, we can add the result (-54078036) to the second number (97075874):\n\n   -54078036 + 97075874 = 42997838\n\nthis matches the first number in the original question, confirming our subtraction is correct.\n\n\n#### ', '-54078036\n']


 96%|█████████▌| 24/25 [02:17<00:05,  5.70s/it]

1841224
['calculation steps:\n\n1. multiply 4697 by 2 (the units digit of 392): 4697 * 2 = 9394\n\n2. multiply 4697 by 90 (the tens digit of 392): 4697 * 90 = 422730\n\n3. multiply 4697 by 300 (the hundreds digit of 392): 4697 * 300 = 1409100\n\n4. add the results from steps 1, 2, and 3: 9394 + 422730 + 1409100 = 1841224\n\nverification:\n\nto verify, we can perform the division 1841224 / 392.\n\n1841224 / 392 = 4697\n\n#### ', '1841224\n']


100%|██████████| 25/25 [02:23<00:00,  5.73s/it]

[1433385.0, 3403660.0, 4264416.0, 91239083923.0, 176308584824.0, -6172832.0, 1207870.0, 4312224.0, 2382366.0, 161356709801.0, 133955823087.0, 20089875.0, 1093742.0, 1411430.0, 7362212.0, 145077692526.0, 139270159325.0, -40269027.0, 5672900.0, 2147612.0, 2218206.0, 37828393205.0, 148502999821.0, -54078036.0, 1841224.0]


In [9]:
accuracy = sum([t == a for t, a in zip(truth[0:25], answers)]) / 25
print(f'prompt accuracy on questions: {accuracy}')


prompt accuracy on questions: 1.0
